In [ ]:
import numpy as np
import pandas as pd
import xlrd
import csv
import nltk



In [ ]:
#convert xlsx to csv
df = pd.read_excel('restaurants-without-IDs.xlsx',index_col=0)  
df.to_csv('restaurants-without-IDs.xlsx', index=False,skip_blank_lines=True)  # index=True to write row index


XLRDError: ignored

In [ ]:
print(df.head(9))

     column2,"column3","column4","column5","column6"
0        name,"address","city","cuisine","unique_id"
1  "103 west"," ""103 w. paces ferry rd."""," ""a...
2  "20 mott"," ""20 mott st.  between bowery and ...
3  "21 club"," ""21 w. 52nd st."""," ""new york""...
4  "2223"," ""2223 market st."""," ""san francisc...
5  "9 jones street"," ""9 jones st."""," ""new yo...
6  "abbey"," ""163 ponce de leon ave."""," ""atla...
7  "abruzzi"," ""2355 peachtree rd.  peachtree ba...
8  "acquarello"," ""1722 sacramento st."""," ""sa...


In [ ]:
#enlever apostrophes tiret trait d'union à la colonne name
import re

irrelevant_regex1 = re.compile(r'[^a-z0-9\s]')
multispace_regex1 = re.compile(r'\s\s+')

def assigner_aucun_symbols_namecolumn(df):
    return df.assign(
        name=df['name']
             .str.replace(irrelevant_regex, ' ')
             .str.replace(multispace_regex, ' '))

df = assigner_aucun_symbols_namecolumn(df)
df.head(9)

KeyError: ignored

In [ ]:
all_addresses = df['addr'].str.cat(df['city'], sep=', ').values
unique_addresses = np.unique(all_addresses)
print(len(all_addresses), len(unique_addresses))

In [ ]:
def assign_postal_lat_lng(df):
    addresses = df['addr'].str.cat(df['city'], sep=', ')
    addresses_to_postal = [address_to_postal.get(a) for a in addresses]
    addresses_to_lat = [address_to_latlng[a][0] if a in address_to_latlng else None for a in addresses]
    addresses_to_lng = [address_to_latlng[a][1] if a in address_to_latlng else None for a in addresses]

    return df.assign(postal=addresses_to_postal, lat=addresses_to_lat, lng=addresses_to_lng)

df = assign_postal_lat_lng(df)
df.head(6)

In [ ]:
#indexage
#use the library recordlinkage, aka Python Record Linkage Toolkit.
#We have the cleaned records, now we want the pairs to compare to find matches.
!pip install recordlinkage
#To produce the pairs, we could do a "full" index, i.e., all records against all records:
import recordlinkage as rl
from recordlinkage.index import Full

full_indexer = Full()
pairs = full_indexer.index(df)
#len(df) * (len(df) - 1) / 2 == 374545
print(f"Full index: {len(df)} records, {len(pairs)} pairs")

Full index: 866 records, 374545 pairs


In [ ]:
indexer = rl.FullIndex()
pairs = indexer.index(df)
print(indexer)

#len(df) * (len(df) - 1) / 2 == 374545
print(len(pairs))

<Full>
374545


In [ ]:
compare_cl = rl.Compare()

In [ ]:
# produce pairs that have equal values for city:
from recordlinkage.index import Block
def SortedNeighbourhood(df, size_window):
  with open(df, "r") as f:
    reader = csv.DictReader(f)
    header = reader.fieldnames
    for countrow in reader:
      city_indexer = Block('city')
      city_index_pairs = address_indexer.index(df)

print(f"city index: {len(city_index_pairs)} pairs")

NameError: ignored

In [ ]:
from recordlinkage.index import SortedNeighbourhood
#producing pairs that have neighboring values for name:
def sortedneighbourhood(df, size_window):
  with open(df, "r") as f:
    reader = csv.DictReader(f)
    header = reader.fieldnames
    for countrow in reader:
      name_indexer = SortedNeighbourhood('name', window=size_window)
      name_index_pairs = name_indexer.index(df)


print(f"Name index: {len(name_index_pairs)} pairs")
sortedneighbourhood(df, 4)

NameError: ignored

In [ ]:
#comparaison phase
#comparison vectors for all indexed pairs, we define a similarity function for each column: fichier Distance_measures_in_Python.ipynb

comp = rl.Compare()
comp.string('name', 'name', method='edit distance', label='name')
comp.string('addr', 'address', method='jaccard distance', label='address')
comp.string('city', 'city', method='nltk distance', label='city')

In [ ]:
#using edit distance, nltk distance, jaccard distance
for s1, s2 in [["arnie morton", "arnie morton s of chicago"],
               ["the palm", "palm the"]]:
    print(f'jarowinkler("{s1}", "{s2}") ==',
          nltk.edit_distance([s1], [s2])[0])
    print(f'levenshtein("{s1}", "{s2}") ==',
          nltk.jaccard_distancen([s1], [s2])[0])
    print(f'lcs("{s1}", "{s2}") ==',
          nltk.tokenize([s1], [s2])[0])
    print()

In [ ]:
comparison_vectors = comp.compute(pairs, df)
comparison_vectors.head(5)

NameError: ignored

In [ ]:
golden_pairs = Block('cluster').index(df_with_truth)
golden_pairs = golden_pairs.swaplevel().sortlevel()[0]
print("Golden pairs:", len(golden_pairs))

In [ ]:
trouve_pairs_set = set(matches.index)

golden_pairs_set = set(golden_pairs)

true_positives = golden_pairs_set & trouve_pairs_set
false_positives = trouve_pairs_set - golden_pairs_set
false_negatives = golden_pairs_set - trouve_pairs_set

print('true_positives total:', len(true_positives))
print('false_positives total:', len(false_positives))
print('false_negatives total:', len(false_negatives))

NameError: ignored